In [1]:
# ======================================================================================================================
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
import matplotlib
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy as cp
# ======================================================================================================================

MinMaxScaler = MinMaxScaler()
random.seed(7777)

matplotlib.rcParams['font.family']='Malgun Gothic'   # 한글 사용
matplotlib.rcParams['axes.unicode_minus'] = False

label = pd.read_csv('train_label.csv')


# ======================================================================================================================


activity = pd.read_csv('train_activity.csv')

# activity acc_id로 groupby
# - 평균을 내지 않은 이유 : 평균을 냈을 경우 캐릭터는 많지만
#   한 캐릭터만으로 활동한 사람의 정보가 과소평가 될 가능성이 있음
activity = activity.groupby(['acc_id'], as_index = False).sum()
activity.drop(columns = ['day','char_id'], inplace = True)
# print(activity.head())



# ======================================================================================================================


combat = pd.read_csv('train_combat.csv')

combat.drop(columns = ['day', 'server', 'char_id', 'class'], inplace = True)
combat_a = combat.groupby(['acc_id'], as_index = False).sum()
combat_a.drop(columns = 'level', inplace = True)

combat_b = combat.groupby(['acc_id'], as_index = False).max()
#  combat.groupby('acc_id', as_index = False).sum().sort_values('acc_id')

# acc_id 기준으로 정리

combat_b = combat_b[['acc_id', 'level']]
combat = pd.merge(combat_b, combat_a, how = 'inner', on = 'acc_id')



# ======================================================================================================================


payment = pd.read_csv('train_payment.csv')

payment = payment.groupby('acc_id', as_index = False).mean()
payment.drop(columns = 'day', inplace = True)

payment.rename(columns = {'amount_spent' : 'amount_spent_pay'}, inplace = True)


# ======================================================================================================================


trade = pd.read_csv('train_trade.csv')

# 거래에 참여한 횟수를 기준으로

# 판매자로서 활동한 acc_id
trade_a = trade.groupby('source_acc_id', as_index = False).count()
trade_a = trade_a[['source_acc_id', 'day']]

# 구매자로서 활동한 acc_id
trade_b = trade.groupby('target_acc_id', as_index = False).count()
trade_b = trade_b[['target_acc_id', 'day']]

x = trade_a['day'].sum() - trade_b['day'].sum()
print(x) # 0

trade_a.rename(columns={'source_acc_id':'acc_id',
                        'day':'sell_item_cnt'}, inplace=True)
trade_b.rename(columns={'target_acc_id':'acc_id',
                        'day':'buy_item_cnt'}, inplace=True)

trade = pd.merge(trade_a, trade_b, how = 'outer', on = 'acc_id')

# 실제 데이터 검색
# trade[trade['source_acc_id'] == 6].count()

# 데이터 검증
# trade_a[trade_a['source_acc_id'] == 6]


# ======================================================================================================================


pledge = pd.read_csv('train_pledge.csv')

ple_1 = pledge.groupby(['server', 'pledge_id', 'day'], as_index = False).mean()
ple_1.drop(columns = ['acc_id', 'char_id', 'day'], inplace = True)
ple_1 = ple_1.groupby(['server', 'pledge_id'], as_index = False).sum()

ple_a = pledge.groupby(['acc_id', 'char_id', 'server', 'pledge_id', 'day'], as_index = False).mean()
ple_a = ple_a.groupby(['acc_id', 'char_id', 'server', 'pledge_id'], as_index = False).sum()
ple_a = ple_a[['acc_id', 'char_id', 'server', 'pledge_id']]

pledge = pd.merge(ple_a, ple_1, how = 'outer', on = ['server', 'pledge_id'])
pledge.drop(columns = ['char_id', 'server', 'pledge_id'], inplace = True)
pledge_total = pledge.groupby(['acc_id'], as_index = False).mean()

pledge_total.rename(columns = {'random_attacker_cnt' : 'random_attacker_cnt_plg',
                              'random_defender_cnt' : 'random_defender_cnt_plg',
                              'same_pledge_cnt' : 'same_pledge_cnt_plg',
                              'temp_cnt' : 'temp_cnt_plg',
                              'etc_cnt':'etc_cnt_plg'}, inplace = True)

# ple_1[(ple_1['pledge_id'] == 25467) & (ple_1['server'] == 'aq')]
# ple_a[(ple_a['pledge_id'] == 25467) & (ple_a['server'] == 'aq')]


# ======================================================================================================================


# label + activity
label_a = pd.merge(label, activity, how = 'outer', on = 'acc_id')

# (label + activity) + combat
label_b = pd.merge(label_a, combat, how = 'outer', on = 'acc_id')

# (label + activity + combat) + payment
label_c = pd.merge(label_b, payment, how = 'outer', on = 'acc_id')

# (label + activity + combat + payment) + trade
label_d = pd.merge(label_c, trade, how = 'outer', on = 'acc_id')
label_d = label_d[label_d['survival_time'] >= 1]

# (label + activity + combat + payment + trade) + pledge_total
label_z = pd.merge(label_d, pledge_total, how = 'outer', on = 'acc_id')

data = label_z.fillna(0)
p = data.corr()

# ======================================================================================================================


0


In [2]:
from IPython.display import display
pd.options.display.max_rows = None

display(p)


,acc_id,survival_time,amount_spent,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,...,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_plg,random_defender_cnt_plg,same_pledge_cnt_plg,temp_cnt_plg,etc_cnt_plg,combat_play_time,non_combat_play_time
acc_id,1.000000,-0.001575,0.005493,-0.002818,0.003232,0.006622,0.000038,-0.000393,0.006187,0.005429,...,-0.000850,0.000610,0.005891,0.000192,-0.006882,0.003075,-0.002015,0.001309,-0.001155,-0.004165
survival_time,-0.001575,1.000000,-0.106241,0.271866,0.161885,-0.097808,-0.102179,-0.073030,0.189934,-0.004625,...,-0.056090,-0.061603,0.058951,0.018084,-0.039739,0.020253,-0.004732,-0.061006,-0.054429,-0.047272
amount_spent,0.005493,-0.106241,1.000000,-0.073406,-0.010778,-0.002585,-0.025621,0.004883,0.011196,-0.005393,...,0.015652,0.021799,0.030398,0.026182,-0.000483,0.029310,0.009280,0.017891,0.001727,-0.029406
playtime,-0.002818,0.271866,-0.073406,1.000000,0.301596,-0.178878,0.047112,-0.163498,-0.065647,0.100395,...,-0.287777,-0.231606,-0.055124,-0.208801,0.064808,-0.222986,-0.180436,-0.170374,-0.017998,0.215695
npc_kill,0.003232,0.161885,-0.010778,0.301596,1.000000,0.039558,-0.026438,-0.057803,0.257203,0.157237,...,0.013115,0.030304,0.123416,0.024971,0.057414,0.042701,0.062798,0.007518,0.174631,0.054713
solo_exp,0.006622,-0.097808,-0.002585,-0.178878,0.039558,1.000000,0.111297,0.596158,0.225991,0.124114,...,0.325269,0.344166,0.019055,0.140470,0.006760,0.167170,0.194109,0.325954,0.169945,-0.055039
party_exp,0.000038,-0.102179,-0.025621,0.047112,-0.026438,0.111297,1.000000,0.131406,-0.015924,0.144956,...,0.014799,0.069534,-0.013890,-0.034131,0.034720,-0.046871,-0.031105,0.099437,0.081824,0.118820
quest_exp,-0.000393,-0.073030,0.004883,-0.163498,-0.057803,0.596158,0.131406,1.000000,0.146883,0.061248,...,0.225832,0.251276,0.008194,0.089280,-0.009648,0.105494,0.130354,0.257679,0.103135,-0.045493
boss_monster,0.006187,0.189934,0.011196,-0.065647,0.257203,0.225991,-0.015924,0.146883,1.000000,0.133730,...,0.202221,0.162366,0.100085,0.141487,0.040507,0.177075,0.135922,0.078846,0.082354,-0.112551
death,0.005429,-0.004625,-0.005393,0.100395,0.157237,0.124114,0.144956,0.061248,0.133730,1.000000,...,0.024182,0.091268,0.145757,0.024415,0.054880,0.039202,0.094774,0.076363,0.084107,0.010400


In [4]:
data

,acc_id,survival_time,amount_spent,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent,amount_spent_pay,sell_item_cnt,buy_item_cnt,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_plg,random_defender_cnt_plg,same_pledge_cnt_plg,temp_cnt_plg,etc_cnt_plg,combat_play_time,non_combat_play_time
0,27835,64.0,0.002559,9.978209,12.747726,0.130439,0.000000,0.022510,1.0,0.245370,0.246819,0.000000,1.199558,0.000000,0.011245,0.0,17.0,0.000000,0.000000,0.000000,1.918911,0.000000,2.139619,1.962575,0.873062,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,12351,64.0,0.120154,82.013292,37.056629,0.532874,0.039412,0.016449,7.0,0.981479,0.987277,0.000000,39.579596,0.000000,1.588348,0.0,17.0,7.124451,0.000000,4.732258,5.756733,0.000000,2.364842,12.854867,0.528062,17.0,9.0,28.402294,6.556226,3.782773,2.804439,1.867763,11.316867,11.085888,3.106572,21.074054,0.019254
2,125437,55.0,0.182593,40.050302,13.461275,0.176408,0.015579,0.133937,7.0,12.023119,12.094146,0.000000,23.991864,0.000000,-0.060990,0.0,17.0,2.631554,0.000000,2.366129,0.000000,0.000000,4.054015,6.770884,1.815358,19.0,57.0,50.380690,29.701510,8.097357,18.106920,3.218910,43.764446,27.291721,35.660301,79.411095,0.000000
3,104483,64.0,0.016241,61.668175,48.607348,0.243065,0.001153,0.006926,2.0,1.472219,1.480916,31.830496,2.044150,0.115662,0.087355,0.0,17.0,0.000000,0.000000,16.562903,1.918911,0.000000,2.139619,2.747605,0.371990,14.0,12.0,74.043254,8.180246,0.369717,4.109830,3.195066,7.426694,17.930793,1.802598,36.301640,0.000000
4,4704,20.0,0.226396,26.002596,17.106278,0.432320,0.545593,0.226839,15.0,0.736109,0.740458,0.000000,9.279142,0.000000,-0.058153,0.0,16.0,1.026948,0.000000,3.549193,3.837822,12.398273,3.603569,5.298953,1.404644,26.0,33.0,6.649457,2.526252,0.100853,0.439763,1.502158,1.094111,7.960085,1.635964,7.578028,0.042944
5,3277,50.0,0.058863,2.073856,0.558694,9.824467,7.183344,18.332398,0.0,2.208328,2.221374,15.915248,0.180208,0.000000,0.000684,0.0,11.0,0.064184,0.000000,0.000000,0.000000,0.000000,1.013504,0.981288,0.383725,2.0,5.0,99.895236,73.261318,15.608266,14.459715,3.576567,12.333617,50.628120,88.379220,87.542434,0.000000
6,118874,64.0,0.030777,10.940478,4.186490,0.060729,0.011616,0.008225,5.0,0.245370,0.246819,0.000000,4.129501,0.000000,0.386815,0.0,16.0,0.000000,0.000000,0.000000,1.918911,0.000000,0.900892,0.785030,0.826123,3.0,7.0,2.381895,0.433072,0.000000,0.000000,0.143063,0.000000,0.543857,0.332678,1.217732,0.621086
7,7678,5.0,0.000000,23.577962,5.766419,53.313968,0.000000,22.137709,3.0,3.435177,3.455470,15.915248,9.443747,0.000000,0.011014,0.0,16.0,0.770211,0.000000,0.000000,2.398639,0.000000,8.783699,8.537202,3.889936,35.0,0.0,9.708027,6.263502,1.263150,2.151743,0.270230,0.530478,3.658673,10.372018,6.686874,0.010451
8,124997,46.0,0.009187,5.086279,8.573381,3.607803,1.536686,0.064935,1.0,0.000000,0.000000,0.000000,1.351956,0.000000,0.001205,0.0,16.0,0.000000,0.000000,1.183064,0.479728,0.000000,0.112612,0.294386,0.035204,24.0,27.0,16.077792,3.166838,0.070119,2.017259,2.038643,1.491970,5.463288,1.548371,8.169563,0.071844
9,116489,23.0,0.000000,1.268015,1.849491,1.839917,1.260736,0.131602,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.022113,0.0,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035204,9.0,18.0,21.978395,3.356307,0.005464,0.516418,1.967112,2.387152,5.240802,0.952027,9.232660,0.018823


In [42]:
# amount = data['amount_spent']
surv = data['survival_time']

In [36]:
data

,acc_id,survival_time,amount_spent,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent,amount_spent_pay,sell_item_cnt,buy_item_cnt,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_plg,random_defender_cnt_plg,same_pledge_cnt_plg,temp_cnt_plg,etc_cnt_plg,combat_play_time,non_combat_play_time
0,27835,64.0,0.002559,9.978209,12.747726,0.130439,0.000000,0.022510,1.0,0.245370,0.246819,0.000000,1.199558,0.000000,0.011245,0.0,17.0,0.000000,0.000000,0.000000,1.918911,0.000000,2.139619,1.962575,0.873062,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,12351,64.0,0.120154,82.013292,37.056629,0.532874,0.039412,0.016449,7.0,0.981479,0.987277,0.000000,39.579596,0.000000,1.588348,0.0,17.0,7.124451,0.000000,4.732258,5.756733,0.000000,2.364842,12.854867,0.528062,17.0,9.0,28.402294,6.556226,3.782773,2.804439,1.867763,11.316867,11.085888,3.106572,21.074054,0.019254
2,125437,55.0,0.182593,40.050302,13.461275,0.176408,0.015579,0.133937,7.0,12.023119,12.094146,0.000000,23.991864,0.000000,-0.060990,0.0,17.0,2.631554,0.000000,2.366129,0.000000,0.000000,4.054015,6.770884,1.815358,19.0,57.0,50.380690,29.701510,8.097357,18.106920,3.218910,43.764446,27.291721,35.660301,79.411095,0.000000
3,104483,64.0,0.016241,61.668175,48.607348,0.243065,0.001153,0.006926,2.0,1.472219,1.480916,31.830496,2.044150,0.115662,0.087355,0.0,17.0,0.000000,0.000000,16.562903,1.918911,0.000000,2.139619,2.747605,0.371990,14.0,12.0,74.043254,8.180246,0.369717,4.109830,3.195066,7.426694,17.930793,1.802598,36.301640,0.000000
4,4704,20.0,0.226396,26.002596,17.106278,0.432320,0.545593,0.226839,15.0,0.736109,0.740458,0.000000,9.279142,0.000000,-0.058153,0.0,16.0,1.026948,0.000000,3.549193,3.837822,12.398273,3.603569,5.298953,1.404644,26.0,33.0,6.649457,2.526252,0.100853,0.439763,1.502158,1.094111,7.960085,1.635964,7.578028,0.042944
5,3277,50.0,0.058863,2.073856,0.558694,9.824467,7.183344,18.332398,0.0,2.208328,2.221374,15.915248,0.180208,0.000000,0.000684,0.0,11.0,0.064184,0.000000,0.000000,0.000000,0.000000,1.013504,0.981288,0.383725,2.0,5.0,99.895236,73.261318,15.608266,14.459715,3.576567,12.333617,50.628120,88.379220,87.542434,0.000000
6,118874,64.0,0.030777,10.940478,4.186490,0.060729,0.011616,0.008225,5.0,0.245370,0.246819,0.000000,4.129501,0.000000,0.386815,0.0,16.0,0.000000,0.000000,0.000000,1.918911,0.000000,0.900892,0.785030,0.826123,3.0,7.0,2.381895,0.433072,0.000000,0.000000,0.143063,0.000000,0.543857,0.332678,1.217732,0.621086
7,7678,5.0,0.000000,23.577962,5.766419,53.313968,0.000000,22.137709,3.0,3.435177,3.455470,15.915248,9.443747,0.000000,0.011014,0.0,16.0,0.770211,0.000000,0.000000,2.398639,0.000000,8.783699,8.537202,3.889936,35.0,0.0,9.708027,6.263502,1.263150,2.151743,0.270230,0.530478,3.658673,10.372018,6.686874,0.010451
8,124997,46.0,0.009187,5.086279,8.573381,3.607803,1.536686,0.064935,1.0,0.000000,0.000000,0.000000,1.351956,0.000000,0.001205,0.0,16.0,0.000000,0.000000,1.183064,0.479728,0.000000,0.112612,0.294386,0.035204,24.0,27.0,16.077792,3.166838,0.070119,2.017259,2.038643,1.491970,5.463288,1.548371,8.169563,0.071844
9,116489,23.0,0.000000,1.268015,1.849491,1.839917,1.260736,0.131602,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.022113,0.0,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035204,9.0,18.0,21.978395,3.356307,0.005464,0.516418,1.967112,2.387152,5.240802,0.952027,9.232660,0.018823


In [44]:
# data.to_csv('./AllMergeFromDonny.csv')
# data = data.drop(columns='amount_spent')
data = data.drop(columns='survival_time')

In [45]:
data = pd.concat([data,surv],axis=1)

In [47]:
data = pd.concat([data,amount],axis=1)

In [48]:
data

,acc_id,playtime,npc_kill,solo_exp,party_exp,quest_exp,boss_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count,level,pledge_cnt,random_attacker_cnt,random_defender_cnt,temp_cnt,same_pledge_cnt,etc_cnt,num_opponent,amount_spent_pay,sell_item_cnt,buy_item_cnt,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt_plg,random_defender_cnt_plg,same_pledge_cnt_plg,temp_cnt_plg,etc_cnt_plg,combat_play_time,non_combat_play_time,survival_time,amount_spent
0,27835,9.978209,12.747726,0.130439,0.000000,0.022510,1.0,0.245370,0.246819,0.000000,1.199558,0.000000,0.011245,0.0,17.0,0.000000,0.000000,0.000000,1.918911,0.000000,2.139619,1.962575,0.873062,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64.0,0.002559
1,12351,82.013292,37.056629,0.532874,0.039412,0.016449,7.0,0.981479,0.987277,0.000000,39.579596,0.000000,1.588348,0.0,17.0,7.124451,0.000000,4.732258,5.756733,0.000000,2.364842,12.854867,0.528062,17.0,9.0,28.402294,6.556226,3.782773,2.804439,1.867763,11.316867,11.085888,3.106572,21.074054,0.019254,64.0,0.120154
2,125437,40.050302,13.461275,0.176408,0.015579,0.133937,7.0,12.023119,12.094146,0.000000,23.991864,0.000000,-0.060990,0.0,17.0,2.631554,0.000000,2.366129,0.000000,0.000000,4.054015,6.770884,1.815358,19.0,57.0,50.380690,29.701510,8.097357,18.106920,3.218910,43.764446,27.291721,35.660301,79.411095,0.000000,55.0,0.182593
3,104483,61.668175,48.607348,0.243065,0.001153,0.006926,2.0,1.472219,1.480916,31.830496,2.044150,0.115662,0.087355,0.0,17.0,0.000000,0.000000,16.562903,1.918911,0.000000,2.139619,2.747605,0.371990,14.0,12.0,74.043254,8.180246,0.369717,4.109830,3.195066,7.426694,17.930793,1.802598,36.301640,0.000000,64.0,0.016241
4,4704,26.002596,17.106278,0.432320,0.545593,0.226839,15.0,0.736109,0.740458,0.000000,9.279142,0.000000,-0.058153,0.0,16.0,1.026948,0.000000,3.549193,3.837822,12.398273,3.603569,5.298953,1.404644,26.0,33.0,6.649457,2.526252,0.100853,0.439763,1.502158,1.094111,7.960085,1.635964,7.578028,0.042944,20.0,0.226396
5,3277,2.073856,0.558694,9.824467,7.183344,18.332398,0.0,2.208328,2.221374,15.915248,0.180208,0.000000,0.000684,0.0,11.0,0.064184,0.000000,0.000000,0.000000,0.000000,1.013504,0.981288,0.383725,2.0,5.0,99.895236,73.261318,15.608266,14.459715,3.576567,12.333617,50.628120,88.379220,87.542434,0.000000,50.0,0.058863
6,118874,10.940478,4.186490,0.060729,0.011616,0.008225,5.0,0.245370,0.246819,0.000000,4.129501,0.000000,0.386815,0.0,16.0,0.000000,0.000000,0.000000,1.918911,0.000000,0.900892,0.785030,0.826123,3.0,7.0,2.381895,0.433072,0.000000,0.000000,0.143063,0.000000,0.543857,0.332678,1.217732,0.621086,64.0,0.030777
7,7678,23.577962,5.766419,53.313968,0.000000,22.137709,3.0,3.435177,3.455470,15.915248,9.443747,0.000000,0.011014,0.0,16.0,0.770211,0.000000,0.000000,2.398639,0.000000,8.783699,8.537202,3.889936,35.0,0.0,9.708027,6.263502,1.263150,2.151743,0.270230,0.530478,3.658673,10.372018,6.686874,0.010451,5.0,0.000000
8,124997,5.086279,8.573381,3.607803,1.536686,0.064935,1.0,0.000000,0.000000,0.000000,1.351956,0.000000,0.001205,0.0,16.0,0.000000,0.000000,1.183064,0.479728,0.000000,0.112612,0.294386,0.035204,24.0,27.0,16.077792,3.166838,0.070119,2.017259,2.038643,1.491970,5.463288,1.548371,8.169563,0.071844,46.0,0.009187
9,116489,1.268015,1.849491,1.839917,1.260736,0.131602,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.022113,0.0,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035204,9.0,18.0,21.978395,3.356307,0.005464,0.516418,1.967112,2.387152,5.240802,0.952027,9.232660,0.018823,23.0,0.000000


In [50]:
data.to_csv('./AllMergeJamDonny.csv')

In [51]:
data.dtypes

acc_id                       int64
playtime                   float64
npc_kill                   float64
solo_exp                   float64
party_exp                  float64
quest_exp                  float64
boss_monster               float64
death                      float64
revive                     float64
exp_recovery               float64
fishing                    float64
private_shop               float64
game_money_change          float64
enchant_count              float64
level                      float64
pledge_cnt                 float64
random_attacker_cnt        float64
random_defender_cnt        float64
temp_cnt                   float64
same_pledge_cnt            float64
etc_cnt                    float64
num_opponent               float64
amount_spent_pay           float64
sell_item_cnt              float64
buy_item_cnt               float64
play_char_cnt              float64
combat_char_cnt            float64
pledge_combat_cnt          float64
random_attacker_cnt_